## checklist

### 진행예정
 1) 추가필요 열: 플랫폼명, 날짜 \
 2) 타 플랫폼과 합쳤을 때 형식 통일하도록 data type과 Nan 처리방법 등 정의 \
 3) 실행소요시간 마지막에 매직커맨드로 체크 \
 4) import는 설치 및 실행하는 거 settings.py 등으로 세팅하기 \
 5) MySQL로 나눠서 관리할 거면, 이름, 주소, 리뷰수 등은 순환주기 하루 어떤가? \
    리뷰수가 request를 한 번 더 보내야 해서 시간을 잡아먹는다 \
    그러면 어차피 스파이더를 두 개로 쪼개야하기는 하겠다
 6) 쌓다가 원하는 호텔 원하는 가격 등 조건 방 나오면 알림 전달 서비스 추가도 좋을듯!


### 완료
 1) page바꿔가면서 했을 때 상품들 연속성 CHK (limit=50에 따른 상품 누락 여부 확인) -> 이상없음. \
 2) page=1, 2 이상 구분할 때 [4], [1]이 아니라 '전체'를 가져오게 해야겠다. 시간 저녁되니까 숫자로 안된다. -> 해결 \
 3) 리뷰수 추가 완료

### 추후 체크 디버깅 예상
 1) 예약만실 시 그대로 반영 or 삭제

In [1]:
import numpy as np
import pandas as pd

In [1]:
# 1. 프로젝트 생성

In [1]:
# !rm -rf goodchoice
!scrapy startproject goodchoice

New Scrapy project 'goodchoice', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/project/crawling/crawling-repo-2/in_progress/goodchoice

You can start your first spider with:
    cd goodchoice
    scrapy genspider example example.com


In [2]:
!tree goodchoice

goodchoice
├── goodchoice
│   ├── __init__.py
│   ├── __pycache__
│   │   ├── __init__.cpython-36.pyc
│   │   ├── items.cpython-36.pyc
│   │   ├── mongodb.cpython-36.pyc
│   │   ├── pipelines.cpython-36.pyc
│   │   └── settings.cpython-36.pyc
│   ├── items.py
│   ├── middlewares.py
│   ├── mongodb.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       ├── __pycache__
│       │   ├── __init__.cpython-36.pyc
│       │   └── spider.cpython-36.pyc
│       └── spider.py
├── goodchoice.csv
└── scrapy.cfg

4 directories, 17 files


In [3]:
# 2. xpath 찾기

In [1]:
# 1) gu별로 페이지 받았을 때, 호텔리스트 받기 위한 xpath

In [ ]:
links = response.xpath('//*[@id="poduct_list_area"]/li/a/@href').extract()

In [ ]:
# 2) 호텔별로 상세페이지 링크 받았을 때, 각호텔별 상세정보 받기 위한 xpath

In [6]:
# xpath 찾기 전에 필요한 사전 import & settings
import scrapy, requests
from scrapy.http import TextResponse
import datetime
import re


KST = datetime.timezone(datetime.timedelta(hours=9))
today = datetime.datetime.now(tz=KST)
start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
end_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # 종료일 디폴트 = 내일
staydays = (datetime.datetime.strptime(end_date,'%Y-%m-%d').date() - \
            datetime.datetime.strptime(start_date,'%Y-%m-%d').date()).days # 숙박일수 디폴트 = 1박start_date, end_date
persons = 2
gu_code = ['2012']


# 2012 하나에 대해 링크들 가져오기
for gu in gu_code:
        url = f'https://www.goodchoice.kr/product/search/2/{gu}?sort=HIT\
&sel_date={start_date}&sel_date2={end_date}&persons={persons}'
        req = requests.get(url)
        response = TextResponse(req.url, body=req.text, encoding='utf-8')
        links = response.xpath('//*[@id="poduct_list_area"]/li/a/@href').extract()

In [22]:
# 2012 링크들 중 3개에 대해서 테스트 진행 -> Success
for link in links[:1]:
    req = requests.get(link)
    response = TextResponse(req.url, body=req.text, encoding='utf-8')
    
    # name
    name = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h2/text()')[0].extract()
    # level
    level = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/span/text()')[0].extract()
    # score
    try:
        score = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/div[1]/span/text()')[0].extract()
    except:
        score = ['점수없음']
    # location    
    location = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/p[2]/text()')[0].extract()
    # roomtype
    try:
        roomtype = response.xpath('//*[@id="product_filter_form"]/article/div/strong/text()')[1:].extract()
    except:
        roomtype = ['예약만실']

    # price
    price = []
    for num in range(2, 2+len(roomtype)):
        try:
            price.append(response.xpath(f'//*[@id="product_filter_form"]/article/div[{num}]/div[3]/div/div/div/p[2]/b/text()')[0].extract())
        except:
            price.append('다른날짜확인')

    # review (link에 직접 담겨있지 않으며 link url의 hotel_id로 request url 재구성 필요)
    hotel_id = re.findall("ano\=([0-9]+)\&", response.url)[0]
    review_url = 'https://www.goodchoice.kr/product/get_review_non'
    params = {"page": 0, "ano": hotel_id}
    response2 = requests.post(review_url, params)
    review = re.findall('"count":([0-9]+)', response2.text)[0]

    # link
    hotel_desc_url = response.url
    
    
    print(name), print(level), print(score), print(location),print(roomtype), print(price), print(review), print(link) 

[공동구매] 인터컨티넨탈 서울 코엑스
5성급
9.6
서울 강남구 삼성동 159
['[주중! 공동구매 머선129 패키지] 수피리어 더블', '[주중! 공동구매 머선129 패키지] 수피리어 트윈', '[주중! 공동구매 머선129 패키지-2인 조식] ...', '[주중! 공동구매 머선129 패키지-2인 조식] ...', '[미니바 습격 패키지] 수피리어 더블', '[미니바 습격 패키지] 수피리어 트윈', '[봄캉스 특가! Fresh한 시작] 수피리어 더블', '[봄캉스 특가! Fresh한 시작] 수피리어 트윈', '수피리어 더블', '수피리어 트윈', '[Stay and Eat more 패키지] 수피리어 더블', '[Stay and Eat more 패키지] 수피리어 트윈', '[술기로운 생활 패키지] 수피리어 더블', '[술기로운 생활 패키지] 수피리어 트윈', '[비타 스프링 패키지] 수피리어 더블', '[비타 스프링 패키지] 수피리어 트윈', '[Lovely Bubbly 키즈 패키지] 수피리어 더블', '[Lovely Bubbly 키즈 패키지] 수피리어 트윈', '[반짝이는 연애일기 패키지] 수피리어 더블', '[반짝이는 연애일기 패키지] 수피리어 트윈']
['119,000원', '119,000원', '159,000원', '159,000원', '185,000원', '185,000원', '202,000원', '202,000원', '211,750원', '211,750원', '212,960원', '212,960원', '220,000원', '220,000원', '230,000원', '230,000원', '250,000원', '250,000원', '260,000원', '260,000원']
261
https://www.goodchoice.kr/product/detail?ano=6980&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12


In [18]:
# 3. items.py

In [2]:
%%writefile goodchoice/goodchoice/items.py
import scrapy


class GoodchoiceItem(scrapy.Item):
    date = scrapy.Field()
    platform = scrapy.Field()
    name = scrapy.Field()
    level = scrapy.Field()
    score = scrapy.Field()
    review_count = scrapy.Field()
    location = scrapy.Field()
    room_type = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Overwriting goodchoice/goodchoice/items.py


In [21]:
# 4. spider.py

In [3]:
%%writefile goodchoice/goodchoice/spiders/spider.py
import scrapy, requests
from scrapy.http import TextResponse
import datetime
import re
from goodchoice.items import GoodchoiceItem


class GoodchoiceSpider(scrapy.Spider):
    name = "Goodchoice"
    allow_domain= ["goodchoice.kr/"]
    KST = datetime.timezone(datetime.timedelta(hours=9))
    today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
    start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
    end_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # 종료일 디폴트 = 내일
    staydays = (datetime.datetime.strptime(end_date,'%Y-%m-%d').date() - \
                datetime.datetime.strptime(start_date,'%Y-%m-%d').date()).days # 숙박일수 디폴트 = 1박start_date, end_date
    persons, staydays = 2, 1
    start_urls= ["https://www.dailyhotel.com/"]

    
    def start_requests(self):
        KST = datetime.timezone(datetime.timedelta(hours=9))
        today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
        start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
        end_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # 종료일 디폴트 = 내일
        persons = 2
        gu_code = ['2012', '2019', '2016', '2014', '2015', '2020', '2018', '2017', '2021']
        for gu in gu_code:
            start_url = f'https://www.goodchoice.kr/product/search/2/{gu}?sort=HIT\
            &sel_date={start_date}&sel_date2={end_date}&persons={persons}'
            yield scrapy.Request(start_url, callback=self.parse)
            
            
    def parse(self, response):
        links = response.xpath('//*[@id="poduct_list_area"]/li/a/@href').extract()
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
            
    def parse_content(self, response):
        KST = datetime.timezone(datetime.timedelta(hours=9))
        today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
        start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
        # date
        date = start_date
        # name
        name = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h2/text()')[0].extract()
        # level
        level = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/span/text()')[0].extract()
        # score
        try:
            tmp = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/div[1]/span/text()')[0].extract()
            score = float(tmp)
        except:
            score = float(0)
        # location    
        location = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/p[2]/text()')[0].extract()
        # roomtype
        try:
            roomtype = response.xpath('//*[@id="product_filter_form"]/article/div/strong/text()')[1:].extract()
        except:
            roomtype = ['예약만실']
        # price
        price = []
        for num in range(2, 2+len(roomtype)):
            try:
                tmp = response.xpath(f'//*[@id="product_filter_form"]/article/div[{num}]/div[3]/div/div/div/p[2]/b/text()')[0].extract()
                tmp = tmp.replace(',',"")
                tmp = tmp.replace('원','')
                price.append(int(tmp))
            except:
                price.append(int(0))
        # review (link에 직접 담겨있지 않으며 link url의 hotel_id로 request url 재구성 필요)
        hotel_id = re.findall("ano\=([0-9]+)\&", response.url)[0]
        review_url = 'https://www.goodchoice.kr/product/get_review_non'
        params = {"page": 0, "ano": hotel_id}
        response2 = requests.post(review_url, params)
        review = re.findall('"count":([0-9]+)', response2.text)[0]
        # link
        hotel_desc_url = response.url
        # platform
        platform = "여기어때"
        
        
        item = GoodchoiceItem()
        for i in range(len(roomtype)):
            item['date'] = date
            item['platform'] = platform
            item['name'] = name
            item['level'] = level
            item['score'] = score
            item['review_count'] = review
            item['location'] = location
            item['room_type'] = roomtype[i]
            item['price'] = price[i]
            item['link'] = hotel_desc_url
            yield item

Writing goodchoice/goodchoice/spiders/spider.py


In [ ]:
# 5. settings.py: robots.txt 여부 파악

In [24]:
# 6. 프로젝트 실행

In [87]:
%pwd

'/home/ubuntu/python3/project/codes/crawling'

In [4]:
%%writefile rungood.sh
cd goodchoice
rm goodchoice.csv
scrapy crawl Goodchoice -o goodchoice.csv 

Writing rungood.sh


In [7]:
# 실행시간 2:33

In [1]:
# 7. pipeline.py: 크롤링데이터 mongodb에 저장

In [2]:
# scrapy project에 pymongo module 생성

In [5]:
%%writefile goodchoice/goodchoice/mongodb.py
import pymongo


client = pymongo.MongoClient("mongodb://15.165.254.37:27017")
collection = client.hotel.hotel_info

Writing goodchoice/goodchoice/mongodb.py


In [1]:
# pipeline.py 설정

In [6]:
%%writefile goodchoice/goodchoice/pipelines.py
from itemadapter import ItemAdapter
from .mongodb import collection


class GoodchoicePipeline:
    def process_item(self, item, spider):
        data = {"date": item['date'], "platform": item['platform'], "name": item['name'],
                "level": item['level'], "score": item['score'], "review": item['review_count'],
                "location": item['location'], "room_type": item['room_type'],
                "price": item['price'], "link": item['link']}
        collection.insert(data)
        return item

Overwriting goodchoice/goodchoice/pipelines.py


In [3]:
# 8. settings.py -> pipeline 설정변경

```
ITEM_PIPELINES = {
   'goodchoice.pipelines.GoodchoicePipeline': 300,
}
```

In [4]:
# 9. 프로젝트 실행!

In [ ]:
!source rungood.sh

In [40]:
# 실행시간 2:05

In [9]:
import pandas as pd
data = pd.read_csv('goodchoice/goodchoice.csv', encoding='utf-8-sig')
print(data.shape)
data.tail()

(2259, 10)


,date,level,link,location,name,platform,price,review_count,room_type,score
2254,2021-03-12,4성급,https://www.goodchoice.kr/product/detail?ano=6...,서울 종로구 관훈동 155-2,나인트리 프리미어 호텔 인사동,여기어때,148000,409,[또또 1+1] 스탠다드 트윈,9.5
2255,2021-03-12,4성급,https://www.goodchoice.kr/product/detail?ano=6...,서울 종로구 관훈동 155-2,나인트리 프리미어 호텔 인사동,여기어때,158000,409,[또또 1+1] 스탠다드 더블,9.5
2256,2021-03-12,부티크,https://www.goodchoice.kr/product/detail?ano=6...,서울 종로구 창신동 383,오마이인 동대문,여기어때,29500,36,스탠다드 싱글,9.5
2257,2021-03-12,부티크,https://www.goodchoice.kr/product/detail?ano=6...,서울 종로구 창신동 383,오마이인 동대문,여기어때,35400,36,스탠다드 트윈,9.5
2258,2021-03-12,부티크,https://www.goodchoice.kr/product/detail?ano=6...,서울 종로구 창신동 383,오마이인 동대문,여기어때,35400,36,스탠다드 더블,9.5


In [ ]:
# 10. crontab 실행